<a href="https://colab.research.google.com/github/arjunparmar/VIRTUON/blob/main/Harshit/CAGAN_Implemetation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import cv2 as cv

%matplotlib inline

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.image import resize
from tensorflow import reshape

In [ ]:
def generator(input, n_filters = 64, k_size = 4, stride = 2):
    
    # Encoder Layer

    # Stage 1
    e1 = layers.Conv2D(n_filters, kernel_size = k_size, strides = stride, padding = "same", name = "Encoder_1")(input)
    n1 = e1
    a1 = layers.LeakyReLU(name = "Encoder_Activation_1")(e1)
    
    # Stage 2
    e2 = layers.Conv2D(n_filters*2, kernel_size = k_size, strides = stride, padding = "same", name = "Encoder_2")(a1)
    n2 = InstanceNormalization(name = "Encoder_Instance_Normalization_2")(e2)
    a2 = layers.LeakyReLU(name = "Encoder_Activation_2")(n2)

    # Stage 3
    e3 = layers.Conv2D(n_filters*4, kernel_size = k_size, strides = stride, padding = "same", name = "Encoder_3")(a2)
    n3 = InstanceNormalization(name = "Encoder_Instance_Normalization_3")(e3)
    a3 = layers.LeakyReLU(name = "Encoder_Activation_3")(n3)

    # Stage 4
    e4 = layers.Conv2D(n_filters*8, kernel_size = k_size, strides = stride, padding = "same", name = "Encoder_4")(a3)
    # n3 = InstanceNormalization(name = "Encoder_Instance_Normalization_4")(e4)
    encoder_output = layers.LeakyReLU(name = "Encoder_Activation_4")(e4)

    # Decoder Layer

    # Stage 1
    d1 = layers.Conv2DTranspose(n_filters*4, kernel_size = k_size, strides = stride, padding = "same", name = "Decoder_1")(encoder_output)
    d1 = InstanceNormalization(name = "Decoder_Instance_Normalization_1")(d1)
    con1 = layers.concatenate([d1,n3])
    a1 = layers.ReLU(name = "Decoder_Activation_1")(con1)

    # Stage 2
    d2 = layers.Conv2DTranspose(n_filters*2, kernel_size = k_size, strides = stride, padding = "same", name = "Decoder_2")(a1)
    d2 = InstanceNormalization(name = "Decoder_Instance_Normalization_2")(d2)
    con2 = layers.concatenate([d2,n2])
    a2 = layers.ReLU(name = "Decoder_Activation_2")(con2)

    # Stage 3
    d3 = layers.Conv2DTranspose(n_filters, kernel_size = k_size, strides = stride, padding = "same", name = "Decoder_3")(a2)
    d3 = InstanceNormalization(name = "Decoder_Instance_Normalization_3")(d3)
    con3 = layers.concatenate([d3,n1])
    decoder_output = layers.ReLU(name = "Decoder_Activation_3")(con3)

    # Output Stage
    output = layers.Conv2DTranspose(4, kernel_size = 2, strides = stride, padding = "same", name = "Output", activation='sigmoid')(decoder_output)

    return Model(input, output, name = "CAGAN Generator")

In [ ]:
test_input = layers.Input((192,256,9), name = "Input")
test = generator(test_input)
# tf.keras.utils.plot_model(test)
test.summary()

Model: "CAGAN Generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 192, 256, 9) 0                                            
__________________________________________________________________________________________________
Encoder_1 (Conv2D)              (None, 96, 128, 64)  9280        Input[0][0]                      
__________________________________________________________________________________________________
Encoder_Activation_1 (LeakyReLU (None, 96, 128, 64)  0           Encoder_1[0][0]                  
__________________________________________________________________________________________________
Encoder_2 (Conv2D)              (None, 48, 64, 128)  131200      Encoder_Activation_1[0][0]       
____________________________________________________________________________________

In [ ]:
def discriminator(input, n_filters = 64, k_size = 4, stride = 2):
    
    x = layers.Conv2D(n_filters, kernel_size=k_size, strides=stride, name = "Layer_64", padding = 'same')(input)
    x = layers.LeakyReLU(name = "Layer_64_Activation")(x)

    x = layers.Conv2D(n_filters*2, kernel_size=k_size, strides=stride, name = "Layer_128", padding = 'same')(x)
    x = layers.BatchNormalization(name = "Layer_128_BatchNormalization")(x)
    x = layers.LeakyReLU(name = "Layer_128_Activation")(x)

    x = layers.Conv2D(n_filters*4, kernel_size=k_size, strides=stride, name = "Layer_256", padding = 'same')(x)
    x = layers.BatchNormalization(name = "Layer_256_BatchNormalization")(x)
    x = layers.LeakyReLU(name = "Layer_256_Activation")(x)

    x = layers.Conv2D(n_filters*8, kernel_size=k_size, strides=1, name = "Layer_512", padding = 'same')(x)
    x = layers.BatchNormalization(name = "Layer_512_BatchNormalization")(x)
    x = layers.LeakyReLU(name = "Layer_512_Activation")(x)

    x = layers.Conv2D(1, kernel_size= 4, strides = 1, name = "Output", padding = 'same')(x)

    return Model(input, x, name = "CAGAN Discriminator")

In [ ]:
check = discriminator(layers.Input((192,256,6), name = "Input"))
check.summary()

Model: "CAGAN Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 192, 256, 6)]     0         
_________________________________________________________________
Layer_64 (Conv2D)            (None, 96, 128, 64)       6208      
_________________________________________________________________
Layer_64_Activation (LeakyRe (None, 96, 128, 64)       0         
_________________________________________________________________
Layer_128 (Conv2D)           (None, 48, 64, 128)       131200    
_________________________________________________________________
Layer_128_BatchNormalization (None, 48, 64, 128)       512       
_________________________________________________________________
Layer_128_Activation (LeakyR (None, 48, 64, 128)       0         
_________________________________________________________________
Layer_256 (Conv2D)           (None, 24, 32, 256